In [ ]:

import abc
from datetime import datetime
from functools import wraps


def log_method_call(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[LOG] Calling method {func.__name__} at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        return func(*args, **kwargs)
    return wrapper



class UniversitySystemError(Exception):
    """Base exception for the university management system."""
    pass


class InvalidIDError(UniversitySystemError):
    """Raised when an invalid ID is provided."""
    pass


class CourseFullError(UniversitySystemError):
    """Raised when a course has reached its maximum capacity."""
    pass



class Person(abc.ABC):
    """
    Abstract base class for all persons in the university system.
    Demonstrates abstraction using the ABC module.
    """
    def __init__(self, name, age, id_number):

        self._name = name
        self._age = age
        self._id_number = id_number


        if not self._validate_id(id_number):
            raise InvalidIDError(f"Invalid ID format: {id_number}")

    @staticmethod
    def _validate_id(id_number):
        """Validate the ID format (protected method)."""

        return isinstance(id_number, str) and id_number.isalnum() and len(id_number) >= 5

    @property
    def name(self):
        """Getter for name attribute."""
        return self._name

    @name.setter
    def name(self, value):
        """Setter for name attribute."""
        if isinstance(value, str) and len(value) > 0:
            self._name = value
        else:
            raise ValueError("Name must be a non-empty string")

    @property
    def age(self):
        """Getter for age attribute."""
        return self._age

    @age.setter
    def age(self, value):
        """Setter for age attribute."""
        if isinstance(value, int) and 16 <= value <= 120:
            self._age = value
        else:
            raise ValueError("Age must be an integer between 16 and 120")

    @property
    def id_number(self):
        """Getter for id_number attribute."""
        return self._id_number

    @abc.abstractmethod
    def get_details(self):
        """
        Abstract method that must be implemented by all subclasses.
        Returns a string with the person's details.
        """
        pass

    @classmethod
    def create_person(cls, person_type, name, age, id_number, **kwargs):
        """
        Factory method to create different types of persons.
        Demonstrates the use of a class method.
        """
        if person_type.lower() == "student":
            return Student(name, age, id_number, **kwargs)
        elif person_type.lower() == "faculty":
            return Faculty(name, age, id_number, **kwargs)
        else:
            raise ValueError(f"Unknown person type: {person_type}")


class Course:
    """Represents a course offered at the university."""
    def __init__(self, course_id, title, credits, max_students=30):
        self._course_id = course_id
        self._title = title
        self._credits = credits
        self._max_students = max_students
        self._enrolled_students = []
        self._instructor = None

    @property
    def course_id(self):
        return self._course_id

    @property
    def title(self):
        return self._title

    @property
    def credits(self):
        return self._credits

    @property
    def instructor(self):
        return self._instructor

    @instructor.setter
    def instructor(self, faculty):
        if isinstance(faculty, Faculty):
            self._instructor = faculty
        else:
            raise TypeError("Instructor must be a Faculty member")

    def add_student(self, student):
        """Add a student to the course."""
        if len(self._enrolled_students) >= self._max_students:
            raise CourseFullError(f"Course {self._title} is full")

        if student not in self._enrolled_students:
            self._enrolled_students.append(student)
            return True
        return False

    def remove_student(self, student):
        """Remove a student from the course."""
        if student in self._enrolled_students:
            self._enrolled_students.remove(student)
            return True
        return False

    def get_enrolled_count(self):
        """Get the number of enrolled students."""
        return len(self._enrolled_students)

    def __str__(self):
        """String representation of the course."""
        return f"Course: {self._title} (ID: {self._course_id}, Credits: {self._credits})"

class Student(Person):
    """Represents a student at the university."""
    def __init__(self, name, age, id_number, major, enrollment_year):

        super().__init__(name, age, id_number)

        self._major = major
        self._enrollment_year = enrollment_year
        self._courses = []
        self._grades = {}

    @property
    def major(self):
        return self._major

    @major.setter
    def major(self, value):
        self._major = value

    @property
    def enrollment_year(self):
        return self._enrollment_year

    @log_method_call
    def enroll_course(self, course):
        """Enroll the student in a course."""
        try:
            if course.add_student(self):
                self._courses.append(course)
                self._grades[course.course_id] = None
                print(f"Student {self._name} enrolled in {course.title}")
                return True
            else:
                print(f"Student {self._name} is already enrolled in {course.title}")
                return False
        except CourseFullError as e:
            print(f"Error: {e}")
            return False

    def assign_grade(self, course_id, grade):
        """Assign a grade to a course."""
        if course_id in self._grades:
            if 0 <= grade <= 100:
                self._grades[course_id] = grade
                return True
            else:
                raise ValueError("Grade must be between 0 and 100")
        else:
            raise ValueError(f"Student is not enrolled in course {course_id}")

    def get_gpa(self):
        """Calculate and return the student's GPA."""
        grades = [g for g in self._grades.values() if g is not None]
        if not grades:
            return 0.0
        return sum(grades) / len(grades)

    def get_details(self):
        """Implement the abstract method from Person (polymorphism)."""
        return (f"Student: {self._name}, ID: {self._id_number}, Age: {self._age}, "
                f"Major: {self._major}, Enrollment Year: {self._enrollment_year}, "
                f"GPA: {self.get_gpa():.2f}")

    def __str__(self):
        """String representation of the student."""
        return self.get_details()


class Faculty(Person):
    """Represents a faculty member at the university."""
    def __init__(self, name, age, id_number, department, rank):

        super().__init__(name, age, id_number)


        self._department = department
        self._rank = rank
        self._courses_taught = []

    @property
    def department(self):
        return self._department

    @department.setter
    def department(self, value):
        self._department = value

    @property
    def rank(self):
        return self._rank

    @rank.setter
    def rank(self, value):
        valid_ranks = ["Assistant Professor", "Associate Professor", "Professor"]
        if value in valid_ranks:
            self._rank = value
        else:
            raise ValueError(f"Invalid rank. Must be one of {valid_ranks}")

    @log_method_call
    def assign_course(self, course):
        """Assign a course to the faculty member."""
        if course not in self._courses_taught:
            self._courses_taught.append(course)
            course.instructor = self
            return True
        return False

    def get_course_load(self):
        """Get the number of courses taught by the faculty member."""
        return len(self._courses_taught)

    def get_details(self):
        """Implement the abstract method from Person (polymorphism)."""
        return (f"Faculty: {self._name}, ID: {self._id_number}, Age: {self._age}, "
                f"Department: {self._department}, Rank: {self._rank}, "
                f"Courses Taught: {len(self._courses_taught)}")

    def __str__(self):
        """String representation of the faculty member."""
        return self.get_details()

class Researcher(abc.ABC):
    """Abstract base class for researchers."""
    def __init__(self, specialization, years_of_experience):
        self._specialization = specialization
        self._years_of_experience = years_of_experience
        self._publications = []

    @property
    def specialization(self):
        return self._specialization

    @property
    def years_of_experience(self):
        return self._years_of_experience

    def add_publication(self, title, journal, year):
        """Add a publication to the researcher's list."""
        publication = {"title": title, "journal": journal, "year": year}
        self._publications.append(publication)

    @abc.abstractmethod
    def get_research_score(self):
        """
        Abstract method to calculate research score.
        Must be implemented by subclasses.
        """
        pass

    def get_publications_count(self):
        """Get the number of publications."""
        return len(self._publications)


class ResearchStudent(Student, Researcher):
    """
    Represents a student who is also a researcher.
    Demonstrates multiple inheritance.
    """
    def __init__(self, name, age, id_number, major, enrollment_year,
                 specialization, years_of_experience, advisor):

        Student.__init__(self, name, age, id_number, major, enrollment_year)
        Researcher.__init__(self, specialization, years_of_experience)

        self._advisor = advisor
        self._thesis_title = None

    @property
    def advisor(self):
        return self._advisor

    @advisor.setter
    def advisor(self, value):
        if isinstance(value, Faculty):
            self._advisor = value
        else:
            raise TypeError("Advisor must be a Faculty member")

    @property
    def thesis_title(self):
        return self._thesis_title

    @thesis_title.setter
    def thesis_title(self, value):
        self._thesis_title = value

    def get_research_score(self):
        """
        Implement the abstract method from Researcher.
        Research score is based on publications and years of experience.
        """
        pub_score = len(self._publications) * 5
        exp_score = self._years_of_experience * 2
        return pub_score + exp_score

    def get_details(self):
        """
        Override the get_details method from Student.
        This demonstrates method overriding (polymorphism).
        """
        student_details = Student.get_details(self)
        return (f"{student_details}, "
                f"Research Specialization: {self._specialization}, "
                f"Research Score: {self.get_research_score()}, "
                f"Thesis: {self._thesis_title or 'Not assigned'}")


class GraduateStudent(Student):
    """
    Represents a graduate student.
    Demonstrates multilevel inheritance (Person -> Student -> GraduateStudent).
    """
    def __init__(self, name, age, id_number, major, enrollment_year, program_type, thesis_topic=None):

        super().__init__(name, age, id_number, major, enrollment_year)

        self._program_type = program_type
        self._thesis_topic = thesis_topic
        self._thesis_advisor = None

    @property
    def program_type(self):
        return self._program_type

    @property
    def thesis_topic(self):
        return self._thesis_topic

    @thesis_topic.setter
    def thesis_topic(self, value):
        self._thesis_topic = value

    @property
    def thesis_advisor(self):
        return self._thesis_advisor

    @thesis_advisor.setter
    def thesis_advisor(self, advisor):
        if isinstance(advisor, Faculty):
            self._thesis_advisor = advisor
        else:
            raise TypeError("Thesis advisor must be a Faculty member")

    def get_details(self):
        """
        Override the get_details method from Student.
        Another example of method overriding (polymorphism).
        """
        student_details = super().get_details()
        advisor_name = self._thesis_advisor.name if self._thesis_advisor else "Not assigned"
        return (f"{student_details}, Program: {self._program_type}, "
                f"Thesis: {self._thesis_topic or 'Not assigned'}, "
                f"Advisor: {advisor_name}")

class PhDResearchStudent(GraduateStudent, Researcher):
    """
    Represents a PhD student who is also a researcher.
    Demonstrates both multiple and multilevel inheritance.
    """
    def __init__(self, name, age, id_number, major, enrollment_year,
                 specialization, years_of_experience, dissertation_topic):

        GraduateStudent.__init__(self, name, age, id_number, major,
                                 enrollment_year, "PhD")
        Researcher.__init__(self, specialization, years_of_experience)

        self._dissertation_topic = dissertation_topic
        self._committee_members = []

    @property
    def dissertation_topic(self):
        return self._dissertation_topic

    @dissertation_topic.setter
    def dissertation_topic(self, value):
        self._dissertation_topic = value

    def add_committee_member(self, faculty):
        """Add a faculty member to the dissertation committee."""
        if isinstance(faculty, Faculty):
            if faculty not in self._committee_members:
                self._committee_members.append(faculty)
                return True
            return False
        else:
            raise TypeError("Committee member must be a Faculty member")

    def get_research_score(self):
        """
        Implement the abstract method from Researcher.
        PhD students get a higher score for publications.
        """
        pub_score = len(self._publications) * 8
        exp_score = self._years_of_experience * 3
        return pub_score + exp_score

    def get_details(self):
        """Override get_details with PhD-specific information."""
        grad_details = GraduateStudent.get_details(self)
        return (f"{grad_details}, "
                f"Research Specialization: {self._specialization}, "
                f"Research Score: {self.get_research_score()}, "
                f"Dissertation: {self._dissertation_topic}, "
                f"Committee Size: {len(self._committee_members)}")


class Department:
    """Represents a department in the university."""
    def __init__(self, name, head=None):
        self._name = name
        self._head = head
        self._faculty_members = []
        self._courses_offered = []

    @property
    def name(self):
        return self._name

    @property
    def head(self):
        return self._head

    @head.setter
    def head(self, faculty):
        if isinstance(faculty, Faculty):
            self._head = faculty
        else:
            raise TypeError("Department head must be a Faculty member")

    def add_faculty(self, faculty):
        """Add a faculty member to the department."""
        if isinstance(faculty, Faculty):
            if faculty not in self._faculty_members:
                self._faculty_members.append(faculty)
                return True
            return False
        else:
            raise TypeError("Must add a Faculty member")

    def add_course(self, course):
        """Add a course to the department's offerings."""
        if course not in self._courses_offered:
            self._courses_offered.append(course)
            return True
        return False

    def get_faculty_count(self):
        """Get the number of faculty members in the department."""
        return len(self._faculty_members)

    def __str__(self):
        """String representation of the department."""
        head_name = self._head.name if self._head else "Not assigned"
        return (f"Department: {self._name}, Head: {head_name}, "
                f"Faculty Count: {self.get_faculty_count()}, "
                f"Courses Offered: {len(self._courses_offered)}")


class University:
    """Main class representing the university."""
    def __init__(self, name):
        self._name = name
        self._departments = {}
        self._students = {}
        self._faculty = {}
        self._courses = {}

    @property
    def name(self):
        return self._name

    def add_department(self, department):
        """Add a department to the university."""
        if department.name not in self._departments:
            self._departments[department.name] = department
            return True
        return False

    def add_student(self, student):
        """Add a student to the university."""
        try:
            if student.id_number not in self._students:
                self._students[student.id_number] = student
                return True
            return False
        except InvalidIDError as e:
            print(f"Error adding student: {e}")
            return False

    def add_faculty(self, faculty):
        """Add a faculty member to the university."""
        try:
            if faculty.id_number not in self._faculty:
                self._faculty[faculty.id_number] = faculty
                return True
            return False
        except InvalidIDError as e:
            print(f"Error adding faculty: {e}")
            return False

    def add_course(self, course):
        """Add a course to the university."""
        if course.course_id not in self._courses:
            self._courses[course.course_id] = course
            return True
        return False

    def get_student(self, student_id):
        """Get a student by ID."""
        try:
            return self._students[student_id]
        except KeyError:
            raise ValueError(f"Student with ID {student_id} not found")

    def get_faculty(self, faculty_id):
        """Get a faculty member by ID."""
        try:
            return self._faculty[faculty_id]
        except KeyError:
            raise ValueError(f"Faculty member with ID {faculty_id} not found")

    def get_course(self, course_id):
        """Get a course by ID."""
        try:
            return self._courses[course_id]
        except KeyError:
            raise ValueError(f"Course with ID {course_id} not found")

    def get_department(self, dept_name):
        """Get a department by name."""
        try:
            return self._departments[dept_name]
        except KeyError:
            raise ValueError(f"Department {dept_name} not found")

    def generate_report(self):
        """Generate a summary report of the university."""
        report = [
            f"University Report: {self._name}",
            f"Number of Departments: {len(self._departments)}",
            f"Number of Faculty: {len(self._faculty)}",
            f"Number of Students: {len(self._students)}",
            f"Number of Courses: {len(self._courses)}",
            "\nDepartments:"
        ]

        for dept in self._departments.values():
            report.append(f"  - {dept}")

        return "\n".join(report)

    def __str__(self):
        """String representation of the university."""
        return f"University: {self._name}"


def main():

    try:
        university = University("Example University")
        print(f"Created {university}")

        cs_dept = Department("Computer Science")
        math_dept = Department("Mathematics")
        physics_dept = Department("Physics")

        university.add_department(cs_dept)
        university.add_department(math_dept)
        university.add_department(physics_dept)

        python_course = Course("CS101", "Introduction to Python", 3)
        java_course = Course("CS102", "Java Programming", 4)
        calculus_course = Course("MTH201", "Calculus I", 4)

        university.add_course(python_course)
        university.add_course(java_course)
        university.add_course(calculus_course)


        cs_dept.add_course(python_course)
        cs_dept.add_course(java_course)
        math_dept.add_course(calculus_course)

        try:
            smith = Faculty("John Smith", 40, "F12345", "Computer Science", "Professor")
            jones = Faculty("Sarah Jones", 35, "F23456", "Mathematics", "Associate Professor")

            university.add_faculty(smith)
            university.add_faculty(jones)

            cs_dept.head = smith
            math_dept.head = jones

            cs_dept.add_faculty(smith)
            math_dept.add_faculty(jones)


            smith.assign_course(python_course)
            smith.assign_course(java_course)
            jones.assign_course(calculus_course)

            alice = Student("Alice Brown", 20, "S12345", "Computer Science", 2023)
            bob = GraduateStudent("Bob Wilson", 24, "S23456", "Mathematics", 2022, "Masters", "Optimization Algorithms")
            charlie = ResearchStudent("Charlie Davis", 25, "S34567", "Computer Science", 2021,
                                     "Machine Learning", 2, smith)
            david = PhDResearchStudent("David Garcia", 28, "S45678", "Physics", 2020,
                                      "Quantum Computing", 3, "Quantum Algorithms for Optimization")

            university.add_student(alice)
            university.add_student(bob)
            university.add_student(charlie)
            university.add_student(david)


            bob.thesis_advisor = jones

            david.add_committee_member(smith)
            david.add_committee_member(jones)

            charlie.add_publication("Machine Learning Applications", "Journal of AI", 2023)
            david.add_publication("Quantum Computing Advances", "Physics Review", 2022)
            david.add_publication("Optimization Algorithms", "Computing Journal", 2023)

            alice.enroll_course(python_course)
            alice.enroll_course(java_course)
            bob.enroll_course(calculus_course)
            charlie.enroll_course(python_course)
            david.enroll_course(calculus_course)


            alice.assign_grade("CS101", 85)
            alice.assign_grade("CS102", 90)
            bob.assign_grade("MTH201", 95)
            charlie.assign_grade("CS101", 92)
            david.assign_grade("MTH201", 98)

            print("\nFaculty Details:")
            print(f"  - {smith}")
            print(f"  - {jones}")

            print("\nStudent Details:")
            print(f"  - {alice}")
            print(f"  - {bob}")
            print(f"  - {charlie}")
            print(f"  - {david}")

            print("\nCourse Details:")
            print(f"  - {python_course}, Instructor: {python_course.instructor.name if python_course.instructor else 'None'}, Enrolled: {python_course.get_enrolled_count()}")
            print(f"  - {java_course}, Instructor: {java_course.instructor.name if java_course.instructor else 'None'}, Enrolled: {java_course.get_enrolled_count()}")
            print(f"  - {calculus_course}, Instructor: {calculus_course.instructor.name if calculus_course.instructor else 'None'}, Enrolled: {calculus_course.get_enrolled_count()}")


            print("\n" + university.generate_report())

        except InvalidIDError as e:
            print(f"ID Error: {e}")
        except ValueError as e:
            print(f"Value Error: {e}")
        except TypeError as e:
            print(f"Type Error: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")


if __name__ == "__main__":
    main()